In [1]:
import auth
from auth import auth_dict

In [10]:
import numpy as np
rangen = np.arange(180, 420)
np.random.choice(rangen)

270

In [2]:
import pandas as pd

In [3]:
spotify = pd.read_csv("SpotifyFeatures.csv")

In [4]:
from datetime import datetime
hour = datetime.now().hour
print("hour =", hour)

def set_parameters(time_of_day, busyness, std_div):
    params = {}

    # Setting parameters for time_of_day...
    if (hour < 6):
         params['mode'] = 'Minor'
         params['tempo'] = (0, 90)
         params['loudness'] = (-33, -10)
    elif (time_of_day < 15):
         params['mode'] = 'Major'
         params['tempo'] = (90, 160)
         params['loudness'] = (-13, -3)
    elif (time_of_day < 21):
         params['mode'] = 'Major'
         params['tempo'] = (130, 230)
         params['loudness'] = (-10, 0)
    else:
         params['mode'] = 'Minor'
         params['tempo'] = (50, 150)
         params['loudness'] = (-16, -12)

    # Setting parameters for busyness...
    if (busyness < 200):
        params['danceability'] = (0.75, 1)
        params['energy'] = (0.8, 1)
    elif (busyness < 1500):
        params['danceability'] = (0.45, 0.75)
        params['energy'] = (0.6, 0.8)
    elif (busyness < 3000):
        params['danceability'] = (0.25, 0.45)
        params['energy'] = (0.4, 0.6)
    else:
        params['danceability'] = (0, 0.3)
        params['energy'] = (0, 0.84)

    # Setting parameters for std_div...
    if (std_div < 5):
        params['valence'] = (0.8, 1)
    elif (std_div < 10):
        params['valence'] = (0.6, 0.8)
    elif (std_div < 15):
        params['valence'] = (0.4, 0.6)
    elif (std_div < 20):
        params['valence'] = (0.2, 0.4)
    else:
        params['valence'] = (0, 0.2)

    return params

#current_params = set_parameters(hour, 86, 2)
#print(current_params)

hour = 0


In [5]:
import requests
occuspace_api = auth_dict["occuspace_api"]
raw_occuspace = requests.get(occuspace_api)

In [6]:
import numpy as np
import json

occuspace_dict = json.loads(raw_occuspace.text)
print(occuspace_dict.keys())

sublocations = occuspace_dict["sublocations"]
# Group different floors to different stress weights
loud = ["1E", "1W", "2E", "2W"]
medium = ["4", "5", "6", "7"]
silent = ["8"]
stress_gauge = 0
# Weight different floors with different stress levels depending on the ideal quietness
for floor in sublocations:
    if floor["abbreviation"] in loud:
        stress_gauge += floor["busyness"] * 1
    elif floor["abbreviation"] in medium:
        stress_gauge += floor["busyness"] * 1.5
    elif floor["abbreviation"] in silent:
        stress_gauge += floor["busyness"] * 2
print("stress_gauge", stress_gauge)
busyness_floors = []

# Calculate standard deviation of all floors' busynesses
for floor in sublocations:
    busyness_floors.append(floor["busyness"])
print("mean", np.mean(busyness_floors))
floors_stdev = np.std(busyness_floors)
print("stdev", floors_stdev)

current_params = set_parameters(hour, stress_gauge, floors_stdev)

dict_keys(['abbreviation', 'isOpen', 'lat', 'locationId', 'locationName', 'lon', 'sublocations', 'units'])
stress_gauge 250.5
mean 27.444444444444443
stdev 32.62620320850205


In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = auth_dict["client_id"]
client_secret = auth_dict["client_secret"]
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

def isExplicit(track_id, client_credentials_manager):
    # generates new access token if needed, otherwise use old one
    client_credentials_manager.get_access_token()
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    isExplicit = bool (sp.track(track_id)["explicit"])
    return isExplicit

In [8]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from auth import auth_dict
import time

def play_song(track_id, track_duration):
    song_url = "https://play.spotify.com/track/{track_id}".format(track_id = track_id)
    login_string = "https://accounts.spotify.com/en/login?continue={song_url}".format(song_url = song_url)
    
    driver = webdriver.Chrome('chromedriver')
    opts = Options()
    opts.add_argument("user-agent=[Mozilla/5.0 (Windows NT x.y; rv:10.0) Gecko/20100101 Firefox/10.0]")
    driver.get(login_string)
    username_form = driver.find_element_by_id("login-username")
    password_form = driver.find_element_by_id("login-password")
    
    username_form.send_keys(auth_dict["username"])
    password_form.send_keys(auth_dict["password"])
    
    login_button = driver.find_element_by_id("login-button")
    login_button.click()
    
    #play_button = driver.find_element_by_class_name("control-button spoticon-play-16 control-button--circled")
    #play_button.click()
    
    time.sleep(track_duration)
    driver.close()

In [ ]:
## pandas conversion
print(current_params["mode"])
no_comedy = spotify[spotify["genre"] != "Comedy"]
matching_mode = no_comedy[spotify["mode"] == current_params["mode"]]
matching_tempo = matching_mode[spotify.tempo.between(current_params["tempo"][0], current_params["tempo"][1])]
matching_loudness = matching_tempo[matching_tempo.loudness.between(current_params["loudness"][0], current_params["loudness"][1])]
matching_danceability = matching_loudness[matching_loudness.danceability.between(current_params["danceability"][0], current_params["danceability"][1])]
matching_valence = matching_danceability[matching_danceability.valence.between(current_params["valence"][0], current_params["valence"][1])]
random_song = matching_valence.sample(1)
random_song
while isExplicit(random_song.track_id.iloc[0], client_credentials_manager):
    random_song = matching_valence.sample(1)
track_id = random_song.track_id.iloc[0]
track_duration = random_song.duration_ms.iloc[0]/1000

print("tempo", random_song["tempo"].iloc[0])
print("mode", random_song["mode"].iloc[0])
print("danceability", random_song["danceability"].iloc[0])
print("valence", random_song["valence"].iloc[0])
play_song(track_id, track_duration)


Minor


C:\Users\Labtop\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\Labtop\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


tempo 70.227
mode Minor
danceability 0.484
valence 0.179
